#**Greetings Generator**

## General Setup and Installation

When you first load the notebook or do a factory reset of the runtime, you will need to install cuda-drivers, python packages and cloudflare

In [ ]:
!sudo apt-get update && sudo apt-get install -y cuda-drivers

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64

In [ ]:
!pip install -U pillow==9.4.0
!pip install opencv-python-headless
!pip install numpy
!pip install pandas
!pip install streamlit

When you restart runtime rather than complete reset, you need to run the below cell to re-download and install ollama again

In [ ]:
!curl https://ollama.ai/install.sh | sh

## Ollama Server Setup

Everytime runtime is restarted, you need to start ollama server

### Setup parallel thread for ollama server to run in the background

In [5]:
import os

os.environ['PATH'] += ':/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

In [6]:
import os
import asyncio

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

### Setup LLM model and system prompt

In [ ]:
!ollama create test -f /content/Modelfile

### Setup Cloudflare for ollama server

Everytime you restart runtime, you will need to setup cloudflare

In [ ]:
!chmod +x cloudflared-linux-amd64
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

In [ ]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

You will use this url to access the UI as well

## Streamlit Setup for UI

In [ ]:
%%writefile interface.py
from io import BytesIO
from PIL import Image
import base64
import streamlit as st
import time
from text2img import process_images_and_serialize
from api import generate
import pandas as pd

greetings = []

st.set_page_config(layout="wide")

page_bg_img = """
<style>
[data-testid="stAppViewContainer"]{
  background-image: url("https://static.vecteezy.com/system/resources/previews/035/320/302/original/2024-dragon-symbol-of-chinese-new-year-free-png.png")
}
</style>
"""

st.markdown(page_bg_img, unsafe_allow_html=True)

st.title('Generate CNY Greetings')

upload_option = st.radio("Choose data input method:", ["Manual Entry", "CSV Upload"])

text_greet, card = st.tabs(["Generate Greeting", "Generate Card"])

def enable():
    st.session_state.running = True

with text_greet:
    if 'generate_button' in st.session_state and st.session_state.generate_button == True:
        st.session_state.running = True
    else:
        st.session_state.running = False

    if 'reset_button' in st.session_state and st.session_state.reset_button == True:
        st.session_state.running1 = False
        st.session_state.reset_button = False
    else:
        st.session_state.running1 = True

    if upload_option == "CSV Upload":
        uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])

        if uploaded_file is not None:
            df = pd.read_csv(uploaded_file)
            st.session_state.num_rows = len(df)

            # Create input fields based on the number of rows in the CSV
            for row in range(st.session_state.num_rows):
                st.session_state[f'Name{str(row+1)}'] = df.iloc[row].get('Name', '')
                st.session_state[f'Content{str(row+1)}'] = df.iloc[row].get('Content', '')
                st.session_state[f'Style{str(row+1)}'] = df.iloc[row].get('Style', '')
                st.session_state[f'Greetings_out{row+1}'] = ''
                st.session_state[f'Greetings{row+1}_image1'] = ''
                st.session_state[f'Greetings{row+1}_image2'] = ''

                row_container = st.empty()
                grid = row_container.columns(3)
                # Columns to lay out the inputs
                with grid[0]:
                    st.text_input(f'Name {row+1}', value = st.session_state[f'Name{str(row+1)}'], key=f'input_col1{row}')
                with grid[1]:
                    st.text_input(f'Content {row+1}',value = st.session_state[f'Content{str(row+1)}'] , key=f'input_col2{row}')
                with grid[2]:
                    st.text_input(f'Style {row+1}',value = st.session_state[f'Style{str(row+1)}'] , key=f'input_col3{row}')

            generated_rows = [f'row {r+1}' for r in range(st.session_state.num_rows)]
    else:
        # Button to add a row
        if st.button('Add Row'):
            st.session_state.num_rows += 1

        if "num_rows" not in st.session_state:
            st.session_state.num_rows = 1

        # Retrieve the number of rows from session state (default to 1 if not yet initialized)
        num_rows = st.session_state.get('num_rows', 1)

        for row in range(st.session_state.num_rows):
            st.session_state[f'Name{str(row+1)}'] = ''
            st.session_state[f'Content{str(row+1)}'] = ''
            st.session_state[f'Style{str(row+1)}'] = ''
            st.session_state[f'Greetings_out{row+1}'] = ''
            st.session_state[f'Greetings{row+1}_image1'] = ''
            st.session_state[f'Greetings{row+1}_image2'] = ''

        def add_row(row):
            row_container = st.empty()
            grid = row_container.columns(3)
            # Columns to lay out the inputs
            with grid[0]:
                st.session_state[f'Name{str(row+1)}'] = st.text_input(f'Name {row+1}', key=f'input_col1{row}')
            with grid[1]:
                st.session_state[f'Content{str(row+1)}'] = st.selectbox(f'Content {row+1}', ("Personal", "Career", "Business"), index=None, placeholder='Please select topic', key=f'input_col2{row}')
            with grid[2]:
                st.session_state[f'Style{str(row+1)}'] = st.selectbox(f'Style {row+1}', ("Formal", "Inspirational", "Funny"), index=None, placeholder='Please select style', key=f'input_col3{row}')
            return

        # Loop to create rows of input widgets
        for r in range(num_rows):
            add_row(r)

        generated_rows = [f'row {r+1}' for r in range(num_rows)]

    # Display the message indicating for which rows greetings are generated

    generate_button = st.button('Generate Greetings', key='generate_button', disabled=st.session_state.running)

    if generate_button:
        with st.spinner('generating greetings...'):
            time.sleep(3)

            # generate greetings for selected rows
            for row in generated_rows:
                output_key = f'output_col1{int(row.split()[1])-1}'
                # replace the following line with your actual greeting generation logic
                #print("hello for " + str(row))
                #output_value = "hello for " + str(row) + st.session_state[f'Name{int(row.split()[1])}'] + "," + st.session_state[f'Company{int(row.split()[1])}']
                output_value = generate(st.session_state[f'Name{int(row.split()[1])}'], st.session_state[f'Content{int(row.split()[1])}'], st.session_state[f'Style{int(row.split()[1])}'])
                if "[Your Name]" in output_value:
                    output_value = output_value.replace("[Your Name]", "Sean")
                st.session_state[f'Greetings_out{int(row.split()[1])}'] = str(output_value)
                st.markdown(f'Greeting {int(row.split()[1])}')
                st.code(st.session_state[f'Greetings_out{int(row.split()[1])}'])
                greetings.append({f'Greetings_out{int(row.split()[1])}': output_value})
                #print(output_value)


            print(greetings)
            #print("greeting is "+ st.session_state['Greetings_out1'])
            # pass greetings to image and get e-cards back as encoded strings
            processed_images_data = process_images_and_serialize(greetings)
            print(len(processed_images_data))
            for enc in processed_images_data:
                for key, value in enc.items():
                    if key.startswith('base64_encoded_image1_greeting') and key[30:]:
                        print(key)
                        j = key[30:]
                        st.session_state[f'Greetings{j}_image1'] = value
                    if key.startswith('base64_encoded_image2_greeting') and key[30:]:
                        j = key[30:]
                        st.session_state[f'Greetings{j}_image2'] = value

            #print("image is "+ st.session_state['Greetings1_image2'])

            # display the message indicating for which rows greetings are generated
        st.success(f'greetings generated for {", ".join(map(str, generated_rows))}.')
        time.sleep(0.5)

with card:
    st.markdown('Current Greetings')

    #print(st.session_state)
    print("greeting is "+ st.session_state['Greetings_out1'])
    for key, value in st.session_state.items():
        if key.startswith('Greetings_out') and key[13:].isdigit():
            print("loop2")
            i = int(key[13:])
            with st.expander(f"Greeting {i}"):
                card_container = st.empty()
                card_grid = card_container.columns(3)
                print(f"{key}: {value}")
                with card_grid[0]:
                    st.markdown('Generated Greeting')
                    st.text(value)
                with card_grid[1]:
                    if st.session_state[f'Greetings{i}_image1'] != '':
                        base64_encoded_image = st.session_state[f'Greetings{i}_image1']
                        # Decode Base64 and recreate the image
                        img_data = base64.b64decode(base64_encoded_image)
                        image_stream = BytesIO(img_data)
                        image = Image.open(image_stream)
                        image.save(f"/content/output_image1_greeting{i}.png", format="PNG")
                        st.image(f"/content/output_image1_greeting{i}.png")
                    else:
                        st.text("card 1 will be here")

                with card_grid[2]:
                    if st.session_state[f'Greetings{i}_image2'] != '':
                        base64_encoded_image = st.session_state[f'Greetings{i}_image2']
                        # Decode Base64 and recreate the image
                        img_data = base64.b64decode(base64_encoded_image)
                        image_stream = BytesIO(img_data)
                        image = Image.open(image_stream)
                        image.save(f"/content/output_image2_greeting{i}.png", format="PNG")
                        st.image(f"/content/output_image2_greeting{i}.png")
                    else:
                        st.text("card 2 will be here")

In [ ]:
%%writefile api.py
import json
import requests

# NOTE: ollama must be running for this to work, start the ollama app or run `ollama serve`
model = 'test' # TODO: update this for whatever model you wish to use

def generate(name, content, style):
    generated_response = ""
    if content == "Business":
        prompt = f'''I want to wish {name} a happy Lunar New Year. Please write a greeting to wish him the best for his business this year. The year is already 2024. It is the year of the dragon too. Only a paragraph of 2 sentences is required. Start with "Dear". Please use a {style} style for the greeting.'''
    if content == "Career":
        prompt = f'''I want to wish {name} a happy Lunar New Year. Please write a greeting to wish him the best for his career this year. The year is already 2024. It is the year of the dragon too. Only a paragraph of 2 sentences is required. Start with "Dear". Please use a {style} style for the greeting.'''
    if content == "Personal":
        prompt = f'''I want to wish {name} a happy Lunar New Year. Please write a greeting to wish him the best for his family life and personal relationships this year. The year is already 2024. It is the year of the dragon too. Only a paragraph of 2 sentences is required. Start with "Dear". Please use a {style} style for the greeting.'''

    print(prompt)

    context = []
    r = requests.post('http://localhost:11434/api/generate',
                      json={
                          'model': model,
                          'prompt': prompt,
                          'context': context,
                      },
                      stream=True)
    r.raise_for_status()

    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        generated_response += response_part
        # the response streams one token at a time, print that as we receive it
        print(response_part, end='', flush=True)

        if 'error' in body:
            raise Exception(body['error'])

        if body.get('done', False):
            context = body['context']

    if "Dear" in generated_response:
        start_index = generated_response.find("Dear")
        generated_response = generated_response[start_index:]
    else:
        generated_response = "Please generate greeting again"
    return generated_response

In [ ]:
%%writefile text2img.py
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import base64
import time

def create_textbox(text, width, height, background_color, font_path, font_color):
    img = Image.new('RGBA', (width, height), background_color)
    draw = ImageDraw.Draw(img)
    font_size = 30
    font = ImageFont.truetype(font_path, font_size)

    # Wrap text to fit within the specified width
    max_text_width = width  # Adjust for padding
    lines = []
    line = ''
    for word in text.split():
        if draw.textsize(line + word, font=font)[0] < max_text_width:
            line += word + ' '
        else:
            lines.append(line)
            line = word + ' '
    lines.append(line)

    # Adjust font size if needed to fit the text within the height
    while sum(draw.textsize(line, font=font)[1] for line in lines) > height:
        font_size -= 1
        font = ImageFont.truetype(font_path, font_size)

    # Draw wrapped text on the image
    y = (height - sum(draw.textsize(line, font=font)[1] for line in lines)) // 2
    for line in lines:
        text_width, text_height = draw.textsize(line, font=font)
        x = (width - text_width) // 2
        draw.text((x, y), line, fill=font_color, font=font)
        y += text_height

    return img

def create_overlay(width, height, background_color):
    img = Image.new('RGBA', (width, height), background_color)
    draw = ImageDraw.Draw(img)
    rectangle_coords = (0, 0, width, height)
    draw.rectangle(rectangle_coords, outline="white", fill=background_color)
    return img

def process_image(input_image, output_image, font_path, greetings, background_color, font_color):
    # Load the image
    image = cv2.imread(input_image)
    original_image = image.copy()

    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the blue color in HSV
    lower_blue = np.array([100, 50, 50])
    upper_blue = np.array([130, 255, 255])

    # Threshold the image to get a binary mask of the blue regions
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours in the grayscale image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through the contours
    for contour in contours:
        # Approximate the contour to determine shape
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

        # If the shape has 4 vertices, it's a rectangle
        if len(approx) == 4:
            # Get bounding box coordinates for the rectangle
            x, y, w, h = cv2.boundingRect(approx)

            # Create a borderless textbox with greetings
            textbox = create_textbox(greetings, w+20, h+10, background_color, font_path, font_color)

            # Convert PIL image to OpenCV format
            textbox_cv = cv2.cvtColor(np.array(textbox), cv2.COLOR_RGBA2BGR)

            # Replace the rectangle with the textbox
            original_image[y-2:y-2+h+10, x-2:x-2+w+20] = textbox_cv

    # Save the modified image
    cv2.imwrite(output_image, original_image)
    print(f"Image with custom text overlay saved as '{output_image}'")

def process_images_and_serialize(greetings):
    processed_images_data = []


    for i in range((len(greetings))):
        # Image 1
        process_image('/content/image1.png', f'/content/image1_edit_greeting{i+1}.png', '/content/Gotham-Black.ttf', greetings[i][f'Greetings_out{i+1}'], '#8B0000', '#F2C96C')
        img1 = cv2.imread(f'image1_edit_greeting{i+1}.png')
        _, img1_encoded = cv2.imencode('.png', img1)
        img1_base64 = base64.b64encode(img1_encoded.tobytes()).decode('utf-8')
        processed_images_data.append({
            f'base64_encoded_image1_greeting{i+1}': img1_base64
        })

        # Image 2
        process_image('/content/image2.png', f'/content/image2_edit_greeting{i+1}.png', '/content/Regular-Brush.ttf', greetings[i][f'Greetings_out{i+1}'], '#790606', '#F2C96C')
        img2 = cv2.imread(f'image2_edit_greeting{i+1}.png')
        _, img2_encoded = cv2.imencode('.png', img2)
        img2_base64 = base64.b64encode(img2_encoded.tobytes()).decode('utf-8')
        processed_images_data.append({
            f'base64_encoded_image2_greeting{i+1}': img2_base64
        })
    return processed_images_data

## Run streamlit

In [15]:
!streamlit run interface.py &>/content/logs.txt &